## Import libraries

In [ ]:
from bs4 import BeautifulSoup
import requests
import os
import pandas 
import re

In [ ]:
cnt = requests.get("https://guide.michelin.com/en/it/restaurants")

C:\Users\flavi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'guide.michelin.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [13]:
soup = BeautifulSoup(cnt.content, features="lxml")
soup.prettify()

'<!DOCTYPE html>\n<html class="full-screen-mobile" dir="" lang="en-US">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="width=device-width, initial-scale=1.0, user-scalable=0" name="viewport"/>\n  <meta content="" name="author"/>\n  <meta content="#fff" name="theme-color"/>\n  <meta content="MICHELIN Guide" property="og:site_name"/>\n  <meta content="Italy MICHELIN Restaurants – The MICHELIN Guide" itemprop="name"/>\n  <meta content="Italy MICHELIN Restaurants – The MICHELIN Guide" property="og:title"/>\n  <meta content="index, follow" name="robots"/>\n  <meta content="Article" property="og:type"/>\n  <meta content="https://guide.michelin.com/en/it/restaurants" property="og:url"/>\n  <meta content="Starred restaurants, Bib Gourmand and all the Restaurants of The MICHELIN Guide Italy. MICHELIN inspector reviews and insights" name="description"/>\n  <meta content="Starred restaurants, Bib Gourmand and all the Restaurants of The MICHELIN Guide Italy. MICHELIN inspector reviews and i

In [ ]:
base_save_path = "E:\Michelin Restaurant"

In [23]:
import time
full_base_url = "https://guide.michelin.com"
base_url="https://guide.michelin.com/en/it/restaurants"
restaurant_links = []
for page_num in range(1,101):
    if page_num%10==0:
        print(f"I'm reading the page number:{page_num}")
    url = f"{base_url}/page/{page_num}"
    response = requests.get(url)
    soup=BeautifulSoup(response.content, features="lxml")
    links = soup.find_all('a', class_="link")  # Adjust 'link' based on actual class name for restaurant links
    # Extract URLs from each link
    for link in links:
        href = link.get('href')
        if "/restaurant/" in href:  # Filter only restaurant links
        # Construct full URL and add to the list
            restaurant_links.append(full_base_url + href)
    time.sleep(2)

I'm reading the page number:10
I'm reading the page number:20
I'm reading the page number:30
I'm reading the page number:40
I'm reading the page number:50
I'm reading the page number:60
I'm reading the page number:70
I'm reading the page number:80
I'm reading the page number:90
I'm reading the page number:100


In [24]:
print(len(restaurant_links))

1983


In [25]:
for link in restaurant_links:
    print(link)

https://guide.michelin.com/en/campania/gragnano/restaurant/o-me-o-il-mare
https://guide.michelin.com/en/abruzzo/popoli_1845563/restaurant/donevandro
https://guide.michelin.com/en/piemonte/alba/restaurant/ape-vino-e-cucina
https://guide.michelin.com/en/campania/sorrento/restaurant/da-bob-cook-fish
https://guide.michelin.com/en/basilicata/matera/restaurant/da-mo
https://guide.michelin.com/en/sardegna/cagliari/restaurant/sa-domu-sarda
https://guide.michelin.com/en/sicilia/palermo/restaurant/charleston
https://guide.michelin.com/en/toscana/bibbiena/restaurant/il-tirabuscio262517
https://guide.michelin.com/en/emilia-romagna/cesenatico/restaurant/la-buca130947
https://guide.michelin.com/en/campania/marina-di-casal-velino/restaurant/alessandro-feo
https://guide.michelin.com/en/lombardia/cervesina/restaurant/dama-1213583
https://guide.michelin.com/en/campania/napoli/restaurant/il-ristorante-alain-ducasse-napoli
https://guide.michelin.com/en/emilia-romagna/noceto_1827072/restaurant/palazzo-utin

In [ ]:
base_save_path = "E:\Michelin Restaurants"

In [34]:
for i in range(0, len(restaurant_links), 20):
    folder_number = (i // 20) + 1
    folder_name = os.path.join(base_save_path, f"page {folder_number}")
    os.makedirs(folder_name, exist_ok=True)
    
    # Estrai il gruppo di URL per questa cartella
    links_to_download = restaurant_links[i:i + 20]
    
    # Scarica ciascun URL e salva il file
    for idx, url in enumerate(links_to_download):
        filename = os.path.join(folder_name, f"restaurant{i + idx + 1}.html")
        response = requests.get(url)
        
        # Salva il contenuto HTML del ristorante nel file
        with open(filename, "w", encoding="utf-8") as file:
            file.write(response.text)

In [79]:
def extract_restaurant_data(file_path):
    # Load and parse the HTML file
    with open(file_path, "r", encoding="utf-8") as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, "html.parser")

    # Initialize the restaurant_info dictionary with default values
    restaurant_info = {
        'restaurantName': None,
        'address': None,
        'city': None,
        'postalCode': None,
        'country': None,
        'priceRange': None,
        'cuisineType': None,
        'description': None,
        'facilitiesServices': [],
        'creditCards': [],
        'phoneNumber': None,
        'website': None
    }

    # Extract Restaurant Name
    title_tag = soup.find('h1', class_='data-sheet__title')
    if title_tag:
        restaurant_info['restaurantName'] = title_tag.get_text(strip=True)

    # Extract Address Components
    address_text = soup.find("div", class_="data-sheet__block--text")
    if address_text:
        address_parts = address_text.text.strip().replace('\n', ' ').split(", ")
        restaurant_info["address"] = address_parts[0] if len(address_parts) > 0 else None
        restaurant_info["city"] = address_parts[1] if len(address_parts) > 1 else None
        restaurant_info["postalCode"] = address_parts[2] if len(address_parts) > 2 else None
        restaurant_info["country"] = address_parts[3] if len(address_parts) > 3 else None

    # Extract Price Range and Cuisine Type
    info_divs = soup.find_all("div", class_="data-sheet__block--text")
    if len(info_divs) > 1:
        info_text = info_divs[1].text.strip()
        if '·' in info_text:
            info_parts = info_text.split('·')
            restaurant_info['priceRange'] = info_parts[0].strip()
            if len(info_parts) > 1:
                restaurant_info['cuisineType'] = info_parts[1].strip()

    # Extract Description
    description_tag = soup.select_one('.data-sheet__description')
    if description_tag:
        restaurant_info['description'] = description_tag.get_text(strip=True)

    # Extract Facilities and Services
    facilities_services = [service.get_text(strip=True) for service in soup.select('.restaurant-details__services li')]
    if facilities_services:
        restaurant_info['facilitiesServices'] = facilities_services

    # Extract Accepted Credit Cards (Modify this part to return a list of credit cards)
    cards=[]
    credit_cards = soup.find_all("img", class_="lazy", height="32")  # Modify as per the HTML structure
    for img in credit_cards:
        src = img.get('src', '') or img.get('data-src', '')
        if src:# Check if 'src' is not empty
            text=src.split("/")[-1]
            cards.append(text.split("-")[0].capitalize())
    restaurant_info["creditCards"]=cards

    # Extract Phone Number
    phone_span = soup.select_one('.collapse__block-item span')
    if phone_span:
        restaurant_info['phoneNumber'] = phone_span.get_text(strip=True)

    # Extract Website URL
    website_link = soup.select_one('.collapse__block-item.link-item a')
    if website_link:
        restaurant_info['website'] = website_link['href']

    return restaurant_info


In [ ]:
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

In [ ]:
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]
def parse_all_folders(base_folder):

    """ Parse all HTML files in a folder structure and aggregate into a DataFrame."""

    data = []  # List to store all parsed data
    
    # Traverse through each folder and file in the base_folder
    for folder_name in sorted(os.listdir(base_folder),key=natural_sort_key):
        folder_path = os.path.join(base_folder, folder_name)
        
        # Check if the path is a directory
        if os.path.isdir(folder_path):
            for filename in sorted(os.listdir(folder_path),key=natural_sort_key):
                # Only process HTML files
                if filename.endswith(".html"):
                    file_path = os.path.join(folder_path, filename)
                    # Parse the file and add to data list
                    restaurant_data = extract_restaurant_data(file_path)
                    data.append(restaurant_data)
    
    # Create DataFrame from collected data
    df = pd.DataFrame(data)
    return df

In [ ]:
# Usage :
base_save_path = "E:\Michelin Restaurants"
df = parse_all_folders(base_save_path)

# Save the DataFrame to a CSV file
df.to_csv("all_restaurants_data.csv", index=False)

In [98]:
df.tail()


,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
1978,Umami,Via Ugo Secondo Partigiano 1,Badalucco,18010,Italy,€€,Modern Cuisine,A young chef with experience in renowned resta...,"[Terrace, Wheelchair access]","[Amex, Mastercard, Visa]",+39 331 338 6005,https://www.umamirestaurant.it/
1979,Visione Restaurant and Living,Strada Nicolini Basso 34,loc. Tre Stelle,Barbaresco,12050,€€€,"Contemporary, Piedmontese","At this restaurant, new, young and enthusiasti...","[Air conditioning, Car park]","[Amex, Maestrocard, Mastercard, Visa]",+39 328 134 0218,https://www.ristorantevisione.it
1980,Ristorante de LEN,Via Cesare Battisti 66,Cortina d'Ampezzo,32043,Italy,€€,Regional Cuisine,Just a stone’s throw from the central and very...,[Wheelchair access],"[Amex, Dinersclub, Mastercard, Visa]",+39 0436 4246,https://hoteldelen.it
1981,AceroRosso,Via Ruvignan 1,Vodo di Cadore,32040,Italy,€€,Regional Cuisine,This secluded mountain chalet immersed in verd...,"[Car park, Terrace, Wheelchair access]","[Mastercard, Visa]",+39 0435 489653,https://www.acerorossodolomiti.it
1982,Café Les Paillotes,piazza Le Laudi 2,Pescara,65129,Italy,€€€,"Modern Cuisine, Seafood",This old acquaintance of the Michelin Guide no...,"[Air conditioning, Interesting wine list, Rest...","[Amex, Dinersclub, Mastercard, Visa]",+39 085 61809,https://www.lespaillotes.it/
